In [1]:
import pandas as pd
import snscrape.modules.twitter as sntwitter #scraping twitter without an API
import datetime

In [3]:
def get_player_tweets(name):
    lst = []
    for i,tweet in enumerate(sntwitter.TwitterSearchScraper(name + ' from:Underdog__NBA since:2022-10-14').get_items()):
        if not "Lineup alert" in tweet.content and not "Load Mismanagement" in tweet.content:
            lst.append([tweet.url, tweet.date, tweet.content])
    df = pd.DataFrame(lst, columns =["URL", "Date", "Text"])
    return df

#Note: to get rewtweeted tweet: tweet.retweetedTweet, and to get quoted: tweet.quotedTweet, to get mentioned users: tweet.mentionedUsers
#to get username: tweet.username

In [84]:
giannis_df = get_player_tweets("Giannis")
giannis_df

C:\Users\16466\AppData\Local\Temp\ipykernel_25484\732506423.py:4: DeprecatedFeatureWarning: content is deprecated, use rawContent instead
  if not "Lineup alert" in tweet.content and not "Load Mismanagement" in tweet.content:
C:\Users\16466\AppData\Local\Temp\ipykernel_25484\732506423.py:5: DeprecatedFeatureWarning: content is deprecated, use rawContent instead
  lst.append([tweet.url, tweet.date, tweet.content])


,URL,Date,Text
0,https://twitter.com/Underdog__NBA/status/16436...,2023-04-05 18:30:28+00:00,Giannis Antetokounmpo (knee) listed probable W...
1,https://twitter.com/Underdog__NBA/status/16411...,2023-03-29 21:38:27+00:00,Giannis Antetokounmpo (knee) will play Wednesday.
2,https://twitter.com/Underdog__NBA/status/16408...,2023-03-28 20:30:48+00:00,Giannis Antetokounmpo (knee) listed probable f...
3,https://twitter.com/Underdog__NBA/status/16404...,2023-03-27 17:30:23+00:00,Giannis Antetokounmpo (knee) listed out Monday.
4,https://twitter.com/Underdog__NBA/status/16400...,2023-03-26 20:31:23+00:00,Giannis Antetokounmpo (knee) listed probable f...
...,...,...,...
104,https://twitter.com/Underdog__NBA/status/15889...,2022-11-05 17:30:30+00:00,Giannis Antetokounmpo (knee) listed questionab...
105,https://twitter.com/Underdog__NBA/status/15887...,2022-11-05 01:30:19+00:00,Giannis Antetokounmpo (knee) listed available ...
106,https://twitter.com/Underdog__NBA/status/15886...,2022-11-05 00:34:11+00:00,Status alert: Giannis Antetokounmpo (knee) a g...
107,https://twitter.com/Underdog__NBA/status/15885...,2022-11-04 17:20:24+00:00,Status alert: Giannis Antetokounmpo (knee) wen...


## Data Cleaning

### Cleaning the Date

1. We need to convert timezone from UTC to US/Eastern.
2. Separate date and time

In [5]:
def clean_date(df):
  """
  Converts the time in the date column to EST, and separates the date and time into two separate columns.
  """
  giannis_df["Date"] = giannis_df["Date"].dt.tz_convert(tz = "US/Eastern") #converting from utc timezone to est
  date = pd.to_datetime(df["Date"])
  df["Date"] = date.dt.date
  df["Time"] = date.dt.time

In [85]:
clean_date(giannis_df)
giannis_df

,URL,Date,Text,Time
0,https://twitter.com/Underdog__NBA/status/16436...,2023-04-05,Giannis Antetokounmpo (knee) listed probable W...,14:30:28
1,https://twitter.com/Underdog__NBA/status/16411...,2023-03-29,Giannis Antetokounmpo (knee) will play Wednesday.,17:38:27
2,https://twitter.com/Underdog__NBA/status/16408...,2023-03-28,Giannis Antetokounmpo (knee) listed probable f...,16:30:48
3,https://twitter.com/Underdog__NBA/status/16404...,2023-03-27,Giannis Antetokounmpo (knee) listed out Monday.,13:30:23
4,https://twitter.com/Underdog__NBA/status/16400...,2023-03-26,Giannis Antetokounmpo (knee) listed probable f...,16:31:23
...,...,...,...,...
104,https://twitter.com/Underdog__NBA/status/15889...,2022-11-05,Giannis Antetokounmpo (knee) listed questionab...,13:30:30
105,https://twitter.com/Underdog__NBA/status/15887...,2022-11-04,Giannis Antetokounmpo (knee) listed available ...,21:30:19
106,https://twitter.com/Underdog__NBA/status/15886...,2022-11-04,Status alert: Giannis Antetokounmpo (knee) a g...,20:34:11
107,https://twitter.com/Underdog__NBA/status/15885...,2022-11-04,Status alert: Giannis Antetokounmpo (knee) wen...,13:20:24


In [7]:
giannis_df["URL"][2]

'https://twitter.com/Underdog__NBA/status/1640813710590595073'

In [8]:
giannis_df[giannis_df["Date"] == datetime.date(2023, 3, 9)] #how to get a tweets from a certain date

,URL,Date,Text,Time
11,https://twitter.com/Underdog__NBA/status/16338...,2023-03-09,Giannis Antetokounmpo (illness) listed out Thu...,13:30:31


### Cleaning the Text of the Tweets

In [9]:
import nltk
from nltk import word_tokenize
import string
from nltk.corpus import stopwords

In [10]:
tweet = giannis_df.loc[giannis_df["Date"] == datetime.date(2023, 3, 10)]["Text"].item()
tweet

'Giannis Antetokounmpo (hand) listed questionable for\xa0Saturday.'

Ideally, the only words in this tweet that we care about is Giannis, Antetokounmpo, hand, questionable, Saturday. <br>
I want to keep the injury type in case I want to look more into it later on. <br>
I am not sure what the \xa0 represents.

#### Removing Punctuation and Stop Words from Tweets

In [11]:
def clean_tweet(tweet):
  """
  Removes puncutation from the tweet, tokenizes it, then filters out any stop words. Returns a LIST of the words in the cleaned tweet
  """
  tweet_no_punc = tweet.translate(str.maketrans("", "",string.punctuation)) #removing puncutation from text by giving a mapping table used to translate the text 
  #needs to be a better way
  word_tokens = word_tokenize(tweet_no_punc) #splitting up the words in the text
  stopwords_custom = set(stopwords.words('english')) 
  stopwords_custom.update(("listed", "status", "alert", "ruled"))
  filtered_tweet = [w for w in word_tokens if not w.lower() in stopwords_custom] #removing all filter words
  return filtered_tweet

In [12]:
def words_to_tweet (words_lst):
  """
  Converts list of words in the tweet back to the tweet itself. Used after cleaning the tweet.
  """
  return ' '.join(words_lst)

In [13]:
def get_clean_tweets(df):
  """
  Creates a new column for the cleaned tweet, then for each tweet, we call clean_tweet and assign it in the new column
  """
  df["Cleaned Text"] = ""
  for ind, tweet in enumerate(df['Text']):
      clean_words_lst = clean_tweet(tweet)
      df["Cleaned Text"][ind] = words_to_tweet(clean_words_lst)
  return df

In [86]:
get_clean_tweets(giannis_df)
giannis_df

,URL,Date,Text,Time,Cleaned Text
0,https://twitter.com/Underdog__NBA/status/16436...,2023-04-05,Giannis Antetokounmpo (knee) listed probable W...,14:30:28,Giannis Antetokounmpo knee probable Wednesday
1,https://twitter.com/Underdog__NBA/status/16411...,2023-03-29,Giannis Antetokounmpo (knee) will play Wednesday.,17:38:27,Giannis Antetokounmpo knee play Wednesday
2,https://twitter.com/Underdog__NBA/status/16408...,2023-03-28,Giannis Antetokounmpo (knee) listed probable f...,16:30:48,Giannis Antetokounmpo knee probable Wednesday
3,https://twitter.com/Underdog__NBA/status/16404...,2023-03-27,Giannis Antetokounmpo (knee) listed out Monday.,13:30:23,Giannis Antetokounmpo knee Monday
4,https://twitter.com/Underdog__NBA/status/16400...,2023-03-26,Giannis Antetokounmpo (knee) listed probable f...,16:31:23,Giannis Antetokounmpo knee probable Monday
...,...,...,...,...,...
104,https://twitter.com/Underdog__NBA/status/15889...,2022-11-05,Giannis Antetokounmpo (knee) listed questionab...,13:30:30,Giannis Antetokounmpo knee questionable Saturday
105,https://twitter.com/Underdog__NBA/status/15887...,2022-11-04,Giannis Antetokounmpo (knee) listed available ...,21:30:19,Giannis Antetokounmpo knee available play Friday
106,https://twitter.com/Underdog__NBA/status/15886...,2022-11-04,Status alert: Giannis Antetokounmpo (knee) a g...,20:34:11,Giannis Antetokounmpo knee gametime decision r...
107,https://twitter.com/Underdog__NBA/status/15885...,2022-11-04,Status alert: Giannis Antetokounmpo (knee) wen...,13:20:24,Giannis Antetokounmpo knee went shootaround Fr...


In [17]:
giannis_df.loc[giannis_df["Date"] == datetime.date(2023, 3, 10)]["Cleaned Text"].item()

'Giannis Antetokounmpo hand questionable Saturday'

#### Finding More Unecessary Words



We removed stopwords, but there are words that I don't really want, such as "listed" and "status". <br>
Let's find the most frequent words. <br>
Note that in function clean_tweet(), I have already added the words that I don't want to appear in the clean tweet versions in the stop words, so when I create a counter of all the words in the tweet, you will not see those words.


In [ ]:
from collections import Counter

In [ ]:
#First, let's get the list of lists of words in the tweets
lst_lst_words = [clean_tweet(tweet) for tweet in giannis_df["Text"]]
#lst_lst_words

In [ ]:
#Then, let's flatten the nested lists to one list
flatten = [word for lst in lst_lst_words for word in lst]
#flatten

In [ ]:
#now, let's count the most common words in the tweet
Counter(flatten).most_common()

[('Giannis', 108),
 ('Antetokounmpo', 107),
 ('knee', 72),
 ('probable', 36),
 ('Monday', 26),
 ('play', 23),
 ('Friday', 19),
 ('Saturday', 16),
 ('available', 15),
 ('Thursday', 15),
 ('Tuesday', 13),
 ('questionable', 10),
 ('wrist', 10),
 ('Sunday', 9),
 ('Wednesday', 7),
 ('remains', 7),
 ('quad', 6),
 ('hand', 5),
 ('gametime', 4),
 ('decision', 4),
 ('remain', 4),
 ('injury', 4),
 ('report', 4),
 ('illness', 3),
 ('return', 3),
 ('calf', 3),
 ('expected', 2),
 ('doubtful', 2),
 ('headed', 2),
 ('locker', 2),
 ('room', 2),
 ('start', 2),
 ('limited', 2),
 ('practice', 2),
 ('MarJon', 2),
 ('Beauchamp', 2),
 ('practices', 2),
 ('went', 2),
 ('shootaround', 2),
 ('ankle', 2),
 ('EbireMoses', 1),
 ('still', 1),
 ('word', 1),
 ('Gannis', 1),
 ('wont', 1),
 ('leg', 1),
 ('diagnosed', 1),
 ('sprained', 1),
 ('ligament', 1),
 ('pain', 1),
 ('swelling', 1),
 ('subside', 1),
 ('undergo', 1),
 ('testing', 1),
 ('injured', 1),
 ('’', 1),
 ('Brook', 1),
 ('Lopez', 1),
 ('2023', 1),
 ('East',

Words we don't like
1. listed
3. status
4. alert
6. ruled

### Convert the references to weekdays in the text to actual dates
Purpose: to match with the dates for betting data


In [18]:
import calendar
from datetime import timedelta

In [19]:
giannis_df.loc[giannis_df["Date"] == datetime.date(2023, 3, 10)]

,URL,Date,Text,Time,Cleaned Text
10,https://twitter.com/Underdog__NBA/status/16342...,2023-03-10,Giannis Antetokounmpo (hand) listed questionab...,14:31:18,Giannis Antetokounmpo hand questionable Saturday


March 10, 2023 was a Friday, so Underdog__NBA simply saying Saturday to refer to a game the next day makes sense. But, in the future, we don't know off the top of our heads that the 3/10/23 is a Friday without looking it up, and we don't know if Saturday might refer to 3/10 or to the next day. Therefore, we need to translate that Saturday into the date that the Bucks play.

My Strategy
1. Take out the weekdays in the tweets and put them into a new column
2. Convert the weekday into number
3. Then we add that to the date of the tweet and we should get date of game being referred to

In [20]:
def extract_days_of_week(text):
  """
  Extracts days of the week from the text by removing the day of the week from the text, then adding the day of the week into a new column. Returns those two columns
  """
  days_of_week = list(calendar.day_name) # words to be removed, days of the week
  words = text.split() # split the text into words
  extracted_words = [w for w in words if w in days_of_week] # extract the words to be removed
  new_text = ' '.join([w for w in words if w not in days_of_week]) # remove the words from the original text
  return pd.Series({'text': new_text, 'extracted_words': ' '.join(extracted_words)}) # return the modified text and extracted words as a Series



In [87]:
#apply the function to every entry in the dataframe
giannis_df[['Cleaned Text', 'Game Weekday']] = giannis_df['Cleaned Text'].apply(extract_days_of_week)


In [89]:
#remove tweets that dont refer to weekdays
giannis_df = giannis_df.drop(giannis_df[giannis_df["Game Weekday"] == ""].index)
giannis_df

,URL,Date,Text,Time,Cleaned Text,Game Weekday
0,https://twitter.com/Underdog__NBA/status/16436...,2023-04-05,Giannis Antetokounmpo (knee) listed probable W...,14:30:28,Giannis Antetokounmpo knee probable,Wednesday
1,https://twitter.com/Underdog__NBA/status/16411...,2023-03-29,Giannis Antetokounmpo (knee) will play Wednesday.,17:38:27,Giannis Antetokounmpo knee play,Wednesday
2,https://twitter.com/Underdog__NBA/status/16408...,2023-03-28,Giannis Antetokounmpo (knee) listed probable f...,16:30:48,Giannis Antetokounmpo knee probable,Wednesday
3,https://twitter.com/Underdog__NBA/status/16404...,2023-03-27,Giannis Antetokounmpo (knee) listed out Monday.,13:30:23,Giannis Antetokounmpo knee,Monday
4,https://twitter.com/Underdog__NBA/status/16400...,2023-03-26,Giannis Antetokounmpo (knee) listed probable f...,16:31:23,Giannis Antetokounmpo knee probable,Monday
...,...,...,...,...,...,...
104,https://twitter.com/Underdog__NBA/status/15889...,2022-11-05,Giannis Antetokounmpo (knee) listed questionab...,13:30:30,Giannis Antetokounmpo knee questionable,Saturday
105,https://twitter.com/Underdog__NBA/status/15887...,2022-11-04,Giannis Antetokounmpo (knee) listed available ...,21:30:19,Giannis Antetokounmpo knee available play,Friday
106,https://twitter.com/Underdog__NBA/status/15886...,2022-11-04,Status alert: Giannis Antetokounmpo (knee) a g...,20:34:11,Giannis Antetokounmpo knee gametime decision r...,Friday
107,https://twitter.com/Underdog__NBA/status/15885...,2022-11-04,Status alert: Giannis Antetokounmpo (knee) wen...,13:20:24,Giannis Antetokounmpo knee went shootaround re...,Friday


In [94]:
def get_game_date(df):
  """
  Get the game date from the game weekday by converting it to an int, converting the tweet weekday to an int
  """
  weekday_map = {'Monday': 0, 'Tuesday': 1, 'Wednesday': 2, 'Thursday': 3, 'Friday': 4, 'Saturday': 5, 'Sunday': 6} # create a mapping of weekday names to numbers
  game_weekday_num = df['Game Weekday'].map(weekday_map).astype(int) # replace weekday names with numbers
  timedelta_series = pd.to_timedelta(game_weekday_num - pd.to_datetime(df['Date']).dt.dayofweek, unit='D') #gives us the difference between the game weekday and tweet weekdat
  df['Game Date'] = pd.to_datetime(df['Date']) + timedelta_series #adds the difference to the tweet date
  df.loc[df["Game Date"] < df["Date"], "Game Date"] += datetime.timedelta(days = 7) #the game date should always be after, so if before add 7
  df['Game Date'] = df['Game Date'].dt.date



I'll give you a scenario for why I have to add 7 in the code above. <br>
The Bucks had a game on November 14, a Monday. In int form, a monday is represented with a 0. <br>
Underdog__NBA tweeted about Giannis' availability on November 13, a Sunday. In int form, a sunday is represented with a 6. <br>
Therefore, game_weekday - tweet_weekday = -6, so the new date would be November 7, but in reality it's November 14. <br>
This scenario only happens when the tweet is on Saturday or Sunday, and the game is on Monday or Tuesday.

In [95]:
get_game_date(giannis_df)
giannis_df

,URL,Date,Text,Time,Cleaned Text,Game Weekday,Game Date
0,https://twitter.com/Underdog__NBA/status/16436...,2023-04-05,Giannis Antetokounmpo (knee) listed probable W...,14:30:28,Giannis Antetokounmpo knee probable,Wednesday,2023-04-05
1,https://twitter.com/Underdog__NBA/status/16411...,2023-03-29,Giannis Antetokounmpo (knee) will play Wednesday.,17:38:27,Giannis Antetokounmpo knee play,Wednesday,2023-03-29
2,https://twitter.com/Underdog__NBA/status/16408...,2023-03-28,Giannis Antetokounmpo (knee) listed probable f...,16:30:48,Giannis Antetokounmpo knee probable,Wednesday,2023-03-29
3,https://twitter.com/Underdog__NBA/status/16404...,2023-03-27,Giannis Antetokounmpo (knee) listed out Monday.,13:30:23,Giannis Antetokounmpo knee,Monday,2023-03-27
4,https://twitter.com/Underdog__NBA/status/16400...,2023-03-26,Giannis Antetokounmpo (knee) listed probable f...,16:31:23,Giannis Antetokounmpo knee probable,Monday,2023-03-27
...,...,...,...,...,...,...,...
104,https://twitter.com/Underdog__NBA/status/15889...,2022-11-05,Giannis Antetokounmpo (knee) listed questionab...,13:30:30,Giannis Antetokounmpo knee questionable,Saturday,2022-11-05
105,https://twitter.com/Underdog__NBA/status/15887...,2022-11-04,Giannis Antetokounmpo (knee) listed available ...,21:30:19,Giannis Antetokounmpo knee available play,Friday,2022-11-04
106,https://twitter.com/Underdog__NBA/status/15886...,2022-11-04,Status alert: Giannis Antetokounmpo (knee) a g...,20:34:11,Giannis Antetokounmpo knee gametime decision r...,Friday,2022-11-04
107,https://twitter.com/Underdog__NBA/status/15885...,2022-11-04,Status alert: Giannis Antetokounmpo (knee) wen...,13:20:24,Giannis Antetokounmpo knee went shootaround re...,Friday,2022-11-04


We are going to get the Bucks

In [ ]:
import nba_api
from nba_api.stats.endpoints import teamgamelog

In [ ]:
bucks_gamelog = teamgamelog.TeamGameLog(team_id=1610612749).get_data_frames()[0]

In [ ]:
bucks_gamelog

,Team_ID,Game_ID,GAME_DATE,MATCHUP,WL,W,L,W_PCT,MIN,FGM,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS
0,1610612749,0022201174,"APR 02, 2023",MIL vs. PHI,W,56,22,0.718,240,46,...,0.714,7,35,42,28,8,5,9,17,117
1,1610612749,0022201146,"MAR 30, 2023",MIL vs. BOS,L,55,22,0.714,240,38,...,0.692,15,30,45,22,4,1,13,16,99
2,1610612749,0022201136,"MAR 29, 2023",MIL @ IND,W,55,21,0.724,240,58,...,0.767,14,38,52,28,5,5,18,18,149
3,1610612749,0022201122,"MAR 27, 2023",MIL @ DET,W,54,21,0.720,240,45,...,0.846,15,35,50,28,5,4,8,23,126
4,1610612749,0022201109,"MAR 25, 2023",MIL @ DEN,L,53,21,0.716,240,39,...,0.905,4,36,40,26,7,2,10,19,106
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73,1610612749,0022200085,"OCT 29, 2022",MIL vs. ATL,W,5,0,1.000,240,45,...,0.655,14,35,49,24,8,5,11,18,123
74,1610612749,0022200075,"OCT 28, 2022",MIL vs. NYK,W,4,0,1.000,240,42,...,0.815,19,41,60,23,8,8,6,23,119
75,1610612749,0022200060,"OCT 26, 2022",MIL vs. BKN,W,3,0,1.000,240,39,...,0.759,14,40,54,21,8,12,18,22,110
76,1610612749,0022200033,"OCT 22, 2022",MIL vs. HOU,W,2,0,1.000,240,48,...,0.619,8,39,47,29,6,10,16,22,125


In [ ]:
def convert_date(gamelog): 
  gamelog["GAME_DATETIME"] = ""
  mon = gamelog["GAME_DATE"].str[0:3]
  day = gamelog["GAME_DATE"].str[4:6]
  yr = gamelog["GAME_DATE"].str[-2:]
  for i in range(len(mon)):
    gamelog["GAME_DATETIME"][i] = datetime.strptime(mon[i] + day[i] + yr[i], "%b%d%y").date()

In [ ]:
convert_date(bucks_gamelog)
bucks_gamelog

<ipython-input-49-0f6692ecc8c1>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gamelog["GAME_DATETIME"][i] = datetime.strptime(mon[i] + day[i] + yr[i], "%b%d%y").date()


,Team_ID,Game_ID,GAME_DATE,MATCHUP,WL,W,L,W_PCT,MIN,FGM,...,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,GAME_DATETIME
0,1610612749,0022201174,"APR 02, 2023",MIL vs. PHI,W,56,22,0.718,240,46,...,7,35,42,28,8,5,9,17,117,2023-04-02
1,1610612749,0022201146,"MAR 30, 2023",MIL vs. BOS,L,55,22,0.714,240,38,...,15,30,45,22,4,1,13,16,99,2023-03-30
2,1610612749,0022201136,"MAR 29, 2023",MIL @ IND,W,55,21,0.724,240,58,...,14,38,52,28,5,5,18,18,149,2023-03-29
3,1610612749,0022201122,"MAR 27, 2023",MIL @ DET,W,54,21,0.720,240,45,...,15,35,50,28,5,4,8,23,126,2023-03-27
4,1610612749,0022201109,"MAR 25, 2023",MIL @ DEN,L,53,21,0.716,240,39,...,4,36,40,26,7,2,10,19,106,2023-03-25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73,1610612749,0022200085,"OCT 29, 2022",MIL vs. ATL,W,5,0,1.000,240,45,...,14,35,49,24,8,5,11,18,123,2022-10-29
74,1610612749,0022200075,"OCT 28, 2022",MIL vs. NYK,W,4,0,1.000,240,42,...,19,41,60,23,8,8,6,23,119,2022-10-28
75,1610612749,0022200060,"OCT 26, 2022",MIL vs. BKN,W,3,0,1.000,240,39,...,14,40,54,21,8,12,18,22,110,2022-10-26
76,1610612749,0022200033,"OCT 22, 2022",MIL vs. HOU,W,2,0,1.000,240,48,...,8,39,47,29,6,10,16,22,125,2022-10-22


Next Steps <br>
1. Make a word cloud (for fun)?
2. add words we don't want to the list

Observations of Data
1. uses weekdays to refer to when a game would be played.
2. sometimes there's weird text

In [ ]:
giannis_df_date.to_csv('giannis_23.csv', sep=',', index=False)